Copyright © 2021, SAS Institute Inc., Cary, NC, USA.  All Rights Reserved.
SPDX-License-Identifier: Apache-2.0

### MLFlow Preamble

In order to import MLFlow models into SAS Model Manager, there are a few lines that need to be included in the MLFlow script. First of all, include the infer_signature function in the import statements:

In [ ]:
from mlflow.models.signature import infer_signature

Next, adjust any data columns which are not valid Python variable names:

In [ ]:
import pandas as pd
data = pd.read_csv('data.csv')
data.columns = data.columns.str.replace('\W|^(?=\d)', '_', regex=True)

Include the signature inference after any paramater logging is defined and include a signature argument in the model logging:

In [ ]:
import mlflow
signature = infer_signature(data, model.predict(data))
mlflow.sklearn.log_model(model, "model", signature=signature)

### Python Package Imports

In [ ]:
# Pathing support
from pathlib import Path

# sasctl interface for importing models
import sasctl.pzmm as pzmm
from sasctl import Session

### Collect MLFlow Model Details

In [ ]:
mlPath = Path('C:/Users/sclind/Documents/Python Scripts/MLFlow/mlflow/examples/sklearn_elasticnet_wine/mlruns/0/7100d4e854224371b39c4a9589317121/artifacts/model')
varDict, inputsDict, outputsDict = pzmm.MLFlowModel.read_mlflow_model_file(_, mlPath)

In [ ]:
outputsDict

### Register Model in SAS Model Manager with pzmm

In [ ]:
modelPrefix = 'MLFlowTest'
zipFolder = Path.cwd() / 'data/MLFlowModels/Test1/'
pzmm.PickleModel.pickle_trained_model(_, modelPrefix, zipFolder, mlflow_details=varDict)

In [ ]:
J = pzmm.JSONFiles()
J.writeVarJSON(inputsDict, isInput=True, jPath=zipFolder)
J.writeVarJSON(outputsDict, isInput=False, jPath=zipFolder)

In [ ]:
# Write model properties to a json file
J.writeModelPropertiesJSON(modelName=modelPrefix,
                            modelDesc='MLFlow Model ',
                            targetVariable='',
                            modelType='',
                            modelPredictors='',
                            targetEvent=1,
                            numTargetCategories=1,
                            eventProbVar='tensor',
                            jPath=zipFolder,
                            modeler='sasdemo')

# Write model metadata to a json file
J.writeFileMetadataJSON(modelPrefix, jPath=zipFolder)

In [ ]:
import getpass
username = getpass.getpass()
password = getpass.getpass()
host = 'demo.sas.com'
sess = Session(host, username, password, protocol='http')

In [ ]:
I = pzmm.ImportModel()
I.pzmmImportModel(zipFolder, modelPrefix, 'MLFlowTest', inputsDict, None, '{}.predict({})', metrics=['tensor'], force=True)